# Introducción

Este script hace la ejecución de Extracción, Procesamiento y Carga (ETL) de los archivos del sitio [Quién es Quién en los precios](https://datos.profeco.gob.mx/datos_abiertos/qqp.php).

Una vez procesados estarán disponibles para la parte A y B del trabajo.

In [1]:
# Library installation of resources not in conda environment "arquitectura"
%pip install awswrangler
%pip install boto3
%pip install rarfile
%pip install selenium
%pip install tqdm
%pip install unidecode

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import libraries
import awswrangler as wr
import boto3
import csv
import matplotlib.pyplot as plt
import os
import pandas as pd
import re
import requests
import seaborn as sns
import subprocess
import time
from bs4 import BeautifulSoup
from tqdm import tqdm
from unidecode import unidecode

### Carga y Preparacion de datos

Descargar los archivos desde el sitio [Quién es Quién en los precios](https://datos.profeco.gob.mx/datos_abiertos/qqp.php) y descomprimirlos en la carpeta `data/`.

In [3]:
# Function to ensure the data directory exists
def ensure_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

# Base URL for making complete links
base_url = "https://datos.profeco.gob.mx/datos_abiertos/"

# URL of the page to scrape
url = "https://datos.profeco.gob.mx/datos_abiertos/qqp.php"

# Ensure the data directory exists
ensure_dir('data')

# Send HTTP GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all <li> tags that contain <a> tags within them
    links = soup.find_all('li')
    file_links = []
    
    for link in tqdm(links):
        a_tag = link.find('a', href=True)
        if a_tag and 'file.php?t=' in a_tag['href']:
            # Create the complete URL for the link
            complete_url = base_url + a_tag['href']
            file_links.append((complete_url, a_tag.text))

    # Visit each link and download the file
    for file_link, name in tqdm(file_links):
        try:
            # Make the request
            response = requests.get(file_link)
            # Save the content to a file
            if response.status_code == 200:
                file_path = os.path.join('data', name.replace('/', '_') + '.rar')  # Replace slashes just in case
                with open(file_path, 'wb') as file:
                    file.write(response.content)
                print(f"File saved: {file_path}")
            else:
                print(f"Failed to download the file from {file_link}. Status code: {response.status_code}")
        except Exception as e:
            print(f"An error occurred while downloading {file_link}: {str(e)}")
else:
    print("Failed to retrieve the webpage. Status code:", response.status_code)

 10%|█         | 1/10 [00:03<00:28,  3.20s/it]

File saved: data\Base de Datos Histórica Quién es Quién en los Precios 2024.rar


 20%|██        | 2/10 [00:13<00:57,  7.19s/it]

File saved: data\Base de Datos Histórica Quién es Quién en los Precios 2023.rar


 30%|███       | 3/10 [00:23<01:00,  8.70s/it]

...

File saved: data\Base de Datos Histórica Quién es Quién en los Precios 2017.rar


 90%|█████████ | 9/10 [01:31<00:10, 10.95s/it]

File saved: data\Base de Datos Histórica Quién es Quién en los Precios 2016.rar


100%|██████████| 10/10 [01:43<00:00, 10.38s/it]

File saved: data\Base de Datos Histórica Quién es Quién en los Precios 2015.rar


Desempacar los archivos

In [4]:
def unpack_rar_files(directory):
    # Change this path to your 7-Zip executable if it's not in the PATH
    seven_zip_path = "7z"
    
    # List all files in the given directory
    files = os.listdir(directory)
    
    # Filter for .rar files
    rar_files = [file for file in files if file.endswith('.rar')]
    
    # Extract each .rar file
    for rar in tqdm(rar_files):
        # Construct the full file path
        file_path = os.path.join(directory, rar)
        # Command to extract the files
        command = [seven_zip_path, 'x', file_path, '-o' + directory]
        # Run the command
        subprocess.run(command, check=True)

# Replace 'data' with your directory path if different
unpack_rar_files('data')

100%|██████████| 10/10 [00:49<00:00,  4.93s/it]


Esquema de los archivos

In [5]:
# Define the expected columns based on the provided schema
expected_columns = [
    'producto',
    'presentacion',
    'marca',
    'categoria',
    'catalogo',
    'precio',
    'fecha_registro',
    'cadena_comercial',
    'giro',
    'nombre_comercial',
    'direccion',
    'estado',
    'municipio',
    'latitud',
    'longitud'
]

Directorio que contiene los archivos

In [6]:
# Directory containing the CSV files
data_dir = 'data'  # Adjust this path as needed in your local setup

Encontrar todos los archivos en el directorio "data"

In [7]:
# Function to find all CSV files in directory and subdirectories
def find_csv_files(directory):
    csv_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.csv'):
                csv_files.append(os.path.join(root, file))
    return csv_files

# Get all CSV files
csv_files = find_csv_files(data_dir)

In [8]:
# Function to clean and transform data
def clean_data(data):
    # Remove accents
    for column in data.columns:
        data[column] = data[column].apply(lambda x: unidecode(str(x)) if isinstance(x, str) and pd.notnull(x) else x)
    # Convert to lowercase
    data = data.apply(lambda x: x.str.lower() if x.dtype == "object" else x)
    # Change commas to pipes in 'direccion', considering complex rules
    # Updated regex pattern to avoid syntax error
    data['direccion'] = data['direccion'].apply(
        lambda x: re.sub(r',(?![^"]*"(?:(?:[^"]*"){2})*[^"]*$)', '|', x) if pd.notnull(x) else x)
    return data

# Ensure the 'data_clean' folder exists
clean_folder = 'data_clean'
os.makedirs(clean_folder, exist_ok=True)

# Read each file and process
for file_path in tqdm(csv_files):
    try:
        # Read the file assuming no headers and using the expected columns
        data = pd.read_csv(file_path, header=None, names=expected_columns)

        # Clean and transform data
        cleaned_data = clean_data(data)

        # Ensure filename is included
        cleaned_data['filename'] = os.path.basename(file_path)

        # Ensure filename is the first column
        cleaned_data = cleaned_data[['filename'] + [col for col in expected_columns if col in cleaned_data.columns]]

        # Save the cleaned data to a new gzip file in the 'data_clean' directory
        clean_file_path = os.path.join(clean_folder, os.path.basename(file_path).replace('.csv', '.csv.gz'))
        cleaned_data.to_csv(clean_file_path, index=False, compression='gzip')
        
    except Exception as e:
        print(f"Error processing {file_path}: {str(e)}")

print("Data processing completed.")

100%|██████████| 450/450 [1:01:23<00:00,  8.19s/it]

Data processing completed.


Cargar los archivos a S3

In [9]:
# Initialize a session
session = boto3.Session(profile_name='arquitectura_AWS_ITAM_2024', region_name='us-east-1')

In [10]:
# Create an S3 client from this session
s3 = session.client('s3')

In [11]:
# Folder to take and S3 bucket to upload
folder_path = 'data_clean'
bucket_name = 'mdge-e3-2024'

In [12]:
# Function to upload files to S3|
def upload_files_to_s3(folder_path, bucket_name):
    
    # Iterate over files in the directory
    for filename in tqdm(os.listdir(folder_path)):
        file_path = os.path.join(folder_path, filename)
        
        # Check if it's a file
        if os.path.isfile(file_path):
            # Upload the file
            try:
                s3.upload_file(file_path, bucket_name, filename)
                print(f"Uploaded {filename} to S3 bucket {bucket_name}")
            except Exception as e:
                print(f"Failed to upload {filename}: {str(e)}")

In [13]:
# Upload files
upload_files_to_s3(folder_path, bucket_name)

  0%|          | 1/450 [00:10<1:20:58, 10.82s/it]

Uploaded 01-2023_01.csv.gz to S3 bucket mdge-e3-2024


  0%|          | 2/450 [00:21<1:18:58, 10.58s/it]

Uploaded 01-2023_02.csv.gz to S3 bucket mdge-e3-2024


  1%|          | 3/450 [00:31<1:17:03, 10.34s/it]

...

Uploaded 532020.csv.gz to S3 bucket mdge-e3-2024


100%|█████████▉| 449/450 [58:37<00:04,  4.94s/it]

Uploaded 532021.csv.gz to S3 bucket mdge-e3-2024


100%|██████████| 450/450 [58:44<00:00,  7.83s/it]

Uploaded 532022.csv.gz to S3 bucket mdge-e3-2024
